## ДЗ 2:
### Методы снижения размерности. Квантизация

In [3]:
import ultralytics
from ultralytics import YOLO

In [4]:
import torch
import time
import torch.quantization
from torch import profiler
from torch.profiler import profile, record_function, ProfilerActivity

In [5]:
torch.seed = 42

In [7]:
def get_model_size(model: torch.nn.Module) -> float:
    """return model size im mb"""
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    return (param_size + buffer_size) / 1024**2

In [8]:
def profile_model(model, input_example: torch.Tensor = torch.randn(1, 3, 224, 224)):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    input_example = input_example.to(device)

    # cold run
    for _ in range(3):
        model(input_example)

    with profile(activities=[ProfilerActivity.CPU], profile_memory=True, with_flops=True) as prof:
        with record_function("model_inference"):
            with torch.inference_mode():
                model(input_example)

    profiling_info = prof.key_averages().table(sort_by="cpu_time_total", row_limit=1)
    return profiling_info

### Исходная модель

In [9]:
yolo = YOLO('yolov8x.pt')
model = yolo.model
model.fuse()
model.eval()
print()

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [10]:
print(profile_model(model))
print(f'model size: {get_model_size(model)}')

STAGE:2023-10-09 21:12:01 34686:1733284 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total KFLOPs  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference        -0.77%   -6267.000us       100.00%     815.282ms     815.282ms           0 b     -77.18 Mb             1            --  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 815.282ms

model size: 260.1646728515625


STAGE:2023-10-09 21:12:02 34686:1733284 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-10-09 21:12:02 34686:1733284 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


### Динамическая квантизация

In [13]:
dynamic_quantized_model = torch.quantization.quantize_dynamic(model, dtype=torch.qint8)

In [14]:
print(profile_model(dynamic_quantized_model))
print(f'model size: {get_model_size(dynamic_quantized_model)}')

STAGE:2023-10-09 21:13:27 34686:1733284 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total KFLOPs  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference        -1.52%  -12462.000us       100.00%     820.687ms     820.687ms           0 b     -76.85 Mb             1            --  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 820.687ms

model size: 260.1646728515625


STAGE:2023-10-09 21:13:28 34686:1733284 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-10-09 21:13:28 34686:1733284 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
